In [3]:
import pandas as pd
import joblib

# Paths to files
csv_path = "/content/enriched_routes_train_combined.csv"
model_path = "/content/travel_time_model.pkl"
columns_path = "/content/model_columns.pkl"

# Load data and model
df = pd.read_csv(csv_path)
model = joblib.load(model_path)
model_columns = joblib.load(columns_path)

# Prepare features
df_processed = pd.get_dummies(df)
for col in model_columns:
    if col not in df_processed.columns:
        df_processed[col] = 0
df_processed = df_processed[model_columns]

# Predict
df['predicted_duration'] = model.predict(df_processed)

# Save result
output_path = "/content/predicted_enriched_routes.csv"
df.to_csv(output_path, index=False)

output_path


'/content/predicted_enriched_routes.csv'

In [4]:
import pandas as pd

# Load the dataset
df = pd.read_csv("predicted_enriched_routes.csv")

# Compute actual duration based on mode
df['actual_duration'] = df.apply(
    lambda row: row['duration_in_traffic'] if row['mode'] == 'driving' and not pd.isna(row['duration_in_traffic']) else row['duration_seconds'],
    axis=1
)

# Calculate absolute and relative errors
df['abs_error'] = (df['predicted_duration'] - df['actual_duration']).abs()
df['rel_error'] = df['abs_error'] / df['actual_duration']

# Summary statistics grouped by mode
error_summary = df.groupby('mode')[['abs_error', 'rel_error']].mean().reset_index()

# Display summary
print("Error Summary by Mode:\n", error_summary)

# Optional: Save results to CSV
df.to_csv("duration_comparison_results.csv", index=False)
error_summary.to_csv("error_summary_by_mode.csv", index=False)


Error Summary by Mode:
         mode   abs_error  rel_error
0  bicycling  172.630000   0.299441
1    driving  126.830387  -0.064971
2    transit  118.786437   0.197120
3    walking  632.711000   0.417625
